### Segmenting and Clustering Neighborhoods in Toronto

In [1]:
#reading/scraping the webpage using pandas
import pandas as pd
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [2]:
#intial shape of the dataframe
df.shape

(287, 3)

In [3]:
#indexing the 'Not assigned' Boroughs
indexNames = df[df['Borough']=='Not assigned'].index

In [4]:
#Dropping the Indexed 'Not assigned' Boroughs
df.drop(indexNames, inplace=True)

In [5]:
#shape after dropping 'Not assigned' Boroughs
df.shape

(210, 3)

In [6]:
#Grouping by Postcode
df=df.groupby(['Postcode', 'Borough']) ['Neighbourhood'].apply (list)
df=df.sample(frac=1).reset_index()
df['Neighbourhood']=df['Neighbourhood'].str.join(',')

In [7]:
#finding the 'Not assigned' Neighbourhood 
df.loc[df['Postcode']=='M9A']

,Postcode,Borough,Neighbourhood
28,M9A,Queen's Park,Not assigned


In [8]:
#replacing Neighbourhood value 'Not assigned' with Borough value
df.loc[(df.Postcode == 'M9A'),'Neighbourhood']=df['Borough']

### Replacing not assigned Neighbourhood 

In [9]:
#checking if the Neighbourhood value 'Not assigned' has updated with Borough value for Postcode M9A 
df.loc[df['Postcode']=='M9A']

,Postcode,Borough,Neighbourhood
28,M9A,Queen's Park,Queen's Park


# Result

In [10]:
#shape of the dataframe
df.shape

(103, 3)

### Renaming the column

In [11]:
df2=df.rename(columns={'Postcode': 'Postal Code'})

In [12]:
df2.head()

,Postal Code,Borough,Neighbourhood
0,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
1,M1S,Scarborough,Agincourt
2,M1B,Scarborough,"Rouge,Malvern"
3,M5G,Downtown Toronto,Central Bay Street
4,M3K,North York,"CFB Toronto,Downsview East"


# Longitude & Latitude

In [13]:
path="http://cocl.us/Geospatial_data"
df3=pd.read_csv(path)

In [14]:
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging Dataframes

In [30]:
df_toronto=pd.merge(df2,df3, on='Postal Code')

In [31]:
df_toronto.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
3,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
4,M3K,North York,"CFB Toronto,Downsview East",43.737473,-79.464763
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M3H,North York,"Bathurst Manor,Downsview North,Wilson Heights",43.754328,-79.442259
7,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
8,M5X,Downtown Toronto,"First Canadian Place,Underground city",43.648429,-79.382280
9,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999


In [37]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### Getting the coordinates of Toronto

In [39]:
address = 'Toronto City, Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


### Creating a map of Toronto

In [33]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### Get Venues nearby Toronto Neighbourhoods

In [34]:
CLIENT_ID = 'FPHNSFYCTY5Y5IWMNECLQNNEVLQPNCXMP54FYP0J3JRT0RCC' # your Foursquare ID
CLIENT_SECRET = 'BJSVSP30WL1LV2X1LUFHB12KBZZ1NISKWCBV4JJ3DJNHTRO4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

##### Define the function to get nearby venues

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Clairlea,Golden Mile,Oakridge
Agincourt
Rouge,Malvern
Central Bay Street
CFB Toronto,Downsview East
Christie
Bathurst Manor,Downsview North,Wilson Heights
Berczy Park
First Canadian Place,Underground city
Kingsway Park South West,Mimico NW,The Queensway West,Royal York South West,South of Bloor
Ryerson,Garden District
North Toronto West
Emery,Humberlea
Maryvale,Wexford
Rosedale
Davisville North
Westmount
Humber Bay Shores,Mimico South,New Toronto
York Mills West
Harbourfront East,Toronto Islands,Union Station
East Toronto
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
Upper Rouge
Willowdale South
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Woodbine Heights
Cabbagetown,St. James Town
Queen's Park
Don Mills North
Downsview Central
The Beaches
Woodbine Gardens,Parkview Hill
Dovercourt Village,Dufferin
Guildwood,Morningside,West Hill
Scarborough Village
Newtonbrook,Willowdale
Lawrence Heights,Lawrence Manor
Roselawn
Adelaide,King,Richmond
CN Tower,Bathurst Quay

In [44]:
print(toronto_venues.shape)

(2230, 7)


In [45]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,Dairy Queen,43.710378,-79.290701,Ice Cream Shop
1,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,Warden Ave & St. Clair Ave E,43.712057,-79.281005,Intersection
2,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,TTC Bus #68 Warden,43.711778,-79.279714,Bus Line
3,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,Warden Subway Station,43.711229,-79.279602,Metro Station
4,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,TTC Bus 102 Markham Road,43.711381,-79.279588,Bus Line


In [52]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",4,4,4,4,4,4
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",9,9,9,9,9,9
"Alderwood,Long Branch",8,8,8,8,8,8
"Bathurst Manor,Downsview North,Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56


##### How many unique venues

In [48]:
print(f"There are {len(toronto_venues['Venue Category'].unique())} unique categories.")


There are 269 unique categories.


### Analyzing Each Neighbourhood

In [50]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [53]:
toronto_onehot.shape

(2230, 269)